In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline
import seaborn as sns

from copy import deepcopy


#For random forest for time series:
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
from skforecast.ForecasterAutoregMultiOutput import ForecasterAutoregMultiOutput
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster

#For metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

#Having to import from subfolders of parent directories is a nightmare
import sys
import os
current = os.path.dirname(os.path.realpath("Single-House-Optimization.py"))
parent = os.path.dirname(current)
sys.path.append(parent+"\Functions")

#From Functions folder
from LoadSeries import load_series, moving_average
from Merge import merge
from Battery import Battery
from DPModel import DPModel, DP
from Logic import logic_bat, logic_actions
from Funcs_Logic_DP import (get_price, get_emissions, logic_rollout, action_rollout, pred_logic_rollout, print_price_summary, 
                            logic_series_print, DP_stochastic, policy_rollout)

In [2]:
house = "h16"
prod = load_series("p",house,"prod_"+house,[24,48,168])
cons = load_series("c",house,"cons_"+house,[24,48,168])
cons

,cons_h16,cons_h16-24,cons_h16-48,cons_h16-168,Year,Month,Day,Weekday,Hour
2020-12-29 00:00:00,5.7,5.7,6.9,6.3,2020,12,29,1,0
2020-12-29 01:00:00,5.9,5.4,5.3,4.8,2020,12,29,1,1
2020-12-29 02:00:00,5.0,6.0,4.9,4.8,2020,12,29,1,2
2020-12-29 03:00:00,3.9,6.8,4.4,4.3,2020,12,29,1,3
2020-12-29 04:00:00,3.5,4.0,3.6,4.0,2020,12,29,1,4
...,...,...,...,...,...,...,...,...,...
2023-01-10 16:00:00,6.1,7.6,6.9,5.4,2023,1,10,1,16
2023-01-10 17:00:00,8.3,10.0,9.8,6.2,2023,1,10,1,17
2023-01-10 18:00:00,10.4,9.1,14.5,13.0,2023,1,10,1,18
2023-01-10 19:00:00,9.9,11.1,10.8,10.4,2023,1,10,1,19


In [3]:
merged = merge(house)
merged

,prod_h16,cons_h16,power_yield,SpotPriceDKK,CO2Emission
2020-12-22 00:00:00,0.0,6.3,-6.3,11.090000,60.0
2020-12-22 01:00:00,0.0,4.8,-4.8,-0.150000,58.0
2020-12-22 02:00:00,0.0,4.8,-4.8,-55.880001,65.0
2020-12-22 03:00:00,0.0,4.3,-4.3,-72.919998,67.0
2020-12-22 04:00:00,0.0,4.0,-4.0,-53.720001,76.0
...,...,...,...,...,...
2022-12-31 19:00:00,0.0,8.7,-8.7,498.329987,209.0
2022-12-31 20:00:00,0.0,6.9,-6.9,301.190002,224.0
2022-12-31 21:00:00,0.0,5.3,-5.3,110.730003,253.0
2022-12-31 22:00:00,0.0,5.6,-5.6,73.919998,272.0


In [4]:
N = 24*7
bat_capacity=13
stepsize=2
Start="2022-06-19 00:00:00"
End = pd.date_range(start=Start, periods=N+(24-stepsize), freq="h")[-1]

train_prod = 4000
train_cons = 500

BeforeProd = pd.date_range(end=Start, periods=train_prod+1, freq="h")[0]
BeforeCons = pd.date_range(end=Start, periods=train_cons+1, freq="h")[0]

print(f"Period: From {Start} to {End}")

Period: From 2022-06-19 00:00:00 to 2022-06-26 21:00:00


In [5]:
forecasterProd = ForecasterAutoreg(
                 regressor = RandomForestRegressor(random_state=123,max_depth=12,max_features='log2',n_estimators=412,n_jobs=4),
                 lags      = list(range(1,24+1,1))
                )
forecasterCons = ForecasterAutoreg(
                 regressor = RandomForestRegressor(random_state=123,max_depth=9,max_features='log2',n_estimators=633,n_jobs=4),
                 lags      = list(range(1,12+1,1))
                )

battery_no = Battery(max_capacity=bat_capacity,max_charge=0)
battery    = Battery(max_capacity=bat_capacity,max_charge=7)
DPbat      = Battery(max_capacity=bat_capacity,max_charge=7)

series_no_battery  = pd.DataFrame(columns=merged.columns)
series_battery     = pd.DataFrame(columns=merged.columns)
series_battery_DP  = pd.DataFrame(columns=merged.columns)

preds = pd.DataFrame() 
for i in range(N//stepsize):#stepsize has to be even
    _, predProd = backtesting_forecaster(forecaster = forecasterProd,
                                         y          = prod.loc[BeforeProd:End].iloc[i*stepsize:train_prod+i*stepsize+24]['prod_'+house],
                                         exog       = prod.loc[BeforeProd:End].iloc[i*stepsize:train_prod+i*stepsize+24].drop(['prod_'+house], axis=1),
                                         initial_train_size = train_prod,
                                         steps      = 24,
                                         refit      = True,
                                         metric     = 'mean_squared_error',
                                         fixed_train_size   = True,
                                         verbose    = False
                                        )

    _, predCons = backtesting_forecaster(forecaster = forecasterCons,
                                         y          = cons.loc[BeforeCons:End].iloc[i*stepsize:train_cons+i*stepsize+24]['cons_'+house],
                                         exog       = cons.loc[BeforeCons:End].iloc[i*stepsize:train_cons+i*stepsize+24].drop(['cons_'+house], axis=1),
                                         initial_train_size = train_cons,
                                         steps      = 24,
                                         refit      = True,
                                         metric     = 'mean_squared_error',
                                         fixed_train_size   = True,
                                         verbose    = False
                                        )
    pred_i = pd.merge(predProd, predCons, how="outer", left_index=True,right_index=True)
    pred_i = pred_i.rename(columns={"pred_x":"prod_"+house,"pred_y":"cons_"+house})

    pred_i["prod_"+house]=np.array((pred_i["prod_"+house].to_numpy())*10,dtype=int)/10
    pred_i["cons_"+house]=np.array((pred_i["cons_"+house].to_numpy())*10,dtype=int)/10

    pred_i["power_yield"] = pred_i["prod_"+house] - pred_i["cons_"+house]
    
    values = pred_i["power_yield"].to_numpy()
    
    for j in range(len(values)):
        values[j] = f"{values[j]:.1f}"
    
    pred_i["power_yield"] = values
    
    
    pred_i["SpotPriceDKK"] = merged.loc[pred_i.index[0]:pred_i.index[-1]]["SpotPriceDKK"]
    pred_i["CO2Emission"] = merged.loc[pred_i.index[0]:pred_i.index[-1]]["CO2Emission"]
    
    merged_i = merged.loc[Start:End].iloc[i*stepsize:i*stepsize+24]
    print(f"i: {i}, From {merged_i.index[0]} to {merged_i.index[-1]}")
    
    temp_no_bat = pred_logic_rollout(merged_i, pred_i, deepcopy(battery_no), logic_bat)
    temp_bat = pred_logic_rollout(merged_i, pred_i, deepcopy(battery), logic_bat)

    actions =  DP(24,pred_i.index[0],pred_i.index[-1],house,pred_i,deepcopy(DPbat),False,True)
    print()
    
    series_no_battery_i  = action_rollout(merged_i.iloc[:stepsize], battery_no, temp_no_bat[:stepsize])
    series_battery_i     = action_rollout(merged_i.iloc[:stepsize], battery, temp_bat[:stepsize])
    series_battery_DP_i  = action_rollout(merged_i.iloc[:stepsize], DPbat, actions)
    
    
    preds=pd.concat([preds,pred_i[:stepsize]])
    series_no_battery  = pd.concat([series_no_battery,series_no_battery_i])
    series_battery     = pd.concat([series_battery,series_battery_i])
    series_battery_DP  = pd.concat([series_battery_DP,series_battery_DP_i])

series_no_battery["cost_cummulative"] = series_no_battery["cost"].cumsum(axis=0)
series_battery["cost_cummulative"] = series_battery["cost"].cumsum(axis=0) 
series_battery_DP["cost_cummulative"] = series_battery_DP["cost"].cumsum(axis=0)
series_no_battery["emission_cummulative"] = series_no_battery["emission"].cumsum(axis=0)
series_battery["emission_cummulative"] = series_battery["emission"].cumsum(axis=0) 
series_battery_DP["emission_cummulative"] = series_battery_DP["emission"].cumsum(axis=0) 
preds

i: 0, From 2022-06-19 00:00:00 to 2022-06-19 23:00:00
0  

i: 1, From 2022-06-19 02:00:00 to 2022-06-20 01:00:00
0  

i: 2, From 2022-06-19 04:00:00 to 2022-06-20 03:00:00
0  

i: 3, From 2022-06-19 06:00:00 to 2022-06-20 05:00:00
0  

i: 4, From 2022-06-19 08:00:00 to 2022-06-20 07:00:00
0  

i: 5, From 2022-06-19 10:00:00 to 2022-06-20 09:00:00
0  

i: 6, From 2022-06-19 12:00:00 to 2022-06-20 11:00:00
0  

i: 7, From 2022-06-19 14:00:00 to 2022-06-20 13:00:00
0  

i: 8, From 2022-06-19 16:00:00 to 2022-06-20 15:00:00
0  

i: 9, From 2022-06-19 18:00:00 to 2022-06-20 17:00:00
0  

i: 10, From 2022-06-19 20:00:00 to 2022-06-20 19:00:00
0  

i: 11, From 2022-06-19 22:00:00 to 2022-06-20 21:00:00
0  

i: 12, From 2022-06-20 00:00:00 to 2022-06-20 23:00:00
0  

i: 13, From 2022-06-20 02:00:00 to 2022-06-21 01:00:00
0  

i: 14, From 2022-06-20 04:00:00 to 2022-06-21 03:00:00
0  

i: 15, From 2022-06-20 06:00:00 to 2022-06-21 05:00:00
0  

i: 16, From 2022-06-20 08:00:00 to 2022-06-21 07:0

,prod_h16,cons_h16,power_yield,SpotPriceDKK,CO2Emission
2022-06-19 00:00:00,0.0,6.9,-6.9,1540.390015,137.0
2022-06-19 01:00:00,0.0,6.5,-6.5,1413.410034,102.0
2022-06-19 02:00:00,0.0,4.9,-4.9,1231.300049,146.0
2022-06-19 03:00:00,0.0,4.0,-4.0,1039.969971,155.0
2022-06-19 04:00:00,0.0,3.7,-3.7,897.140015,155.0
...,...,...,...,...,...
2022-06-25 19:00:00,9.0,9.6,-0.6,2336.860107,104.0
2022-06-25 20:00:00,3.7,8.5,-4.8,2434.100098,103.0
2022-06-25 21:00:00,1.4,7.9,-6.5,2400.699951,99.0
2022-06-25 22:00:00,0.2,7.4,-7.2,2410.669922,86.0


In [6]:
print_price_summary(series_no_battery)
logic_series_print(series_no_battery)

The period is from 2022-06-19 00:00:00 to 2022-06-25 23:00:00
Cost for period:  802.0  DKK
Total emissions for period:  64.0  kg
Average cost per year is: 42077.0 DKK
Average emissions per year is: 3354.0 kg
Number of kwh purchased in the period: 456.3
Average number of kwh purchased per year: 23951.651497005987
Average number of kwh sold per year: 34334.3748502994
hour     price    eprice   yield    surplus  buy      charge   before   degrade  after    cost     pcumsum  emis     ecumsum 
    0:   1.5404,  0.1370, -5.5000, -5.5000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  8.4721,  8.4721,  0.7535,  0.7535
    1:   1.4134,  0.1020, -5.3000, -5.3000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  7.4911, 15.9632,  0.5406,  1.2941
    2:   1.2313,  0.1460, -4.2000, -4.2000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  5.1715, 21.1347,  0.6132,  1.9073
    3:   1.0400,  0.1550, -3.8000, -3.8000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  3.9519, 25.0866,  0.5890,  2.4963
    4:   0

  101:   2.0806,  0.1250, -3.6000, -3.6000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  7.4901,549.0257,  0.4500, 46.4840
  102:   2.5003,  0.1290, -2.9000, -2.9000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  7.2508,556.2766,  0.3741, 46.8581
  103:   2.7030,  0.1450, -2.4000, -2.4000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  6.4872,562.7638,  0.3480, 47.2061
  104:   2.6066,  0.1720, -1.1000, -1.1000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  2.8672,565.6310,  0.1892, 47.3953
  105:   2.3525,  0.1580,  2.1000,  2.1000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -0.4940,565.1370,  0.0000, 47.3953
  106:   2.0668,  0.1490,  5.8000,  5.8000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -1.1987,563.9383,  0.0000, 47.3953
  107:   1.8326,  0.1370, 11.0000, 11.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -2.0158,561.9225,  0.0000, 47.3953
  108:   1.7906,  0.1120, 16.7000, 16.7000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -2.9903,558.9322,  0.0000, 47.3953
  109:  

In [7]:
print_price_summary(series_battery)
logic_series_print(series_battery)

The period is from 2022-06-19 00:00:00 to 2022-06-25 23:00:00
Cost for period:  646.0  DKK
Total emissions for period:  58.0  kg
Average cost per year is: 33923.0 DKK
Average emissions per year is: 3033.0 kg
Number of kwh purchased in the period: 403.5
Average number of kwh purchased per year: 21180.125748502993
Average number of kwh sold per year: 31552.350898203593
hour     price    eprice   yield    surplus  buy      charge   before   degrade  after    cost     pcumsum  emis     ecumsum 
    0:   1.5404,  0.1370, -5.5000, -5.5000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  8.4721,  8.4721,  0.7535,  0.7535
    1:   1.4134,  0.1020, -5.3000, -5.3000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  7.4911, 15.9632,  0.5406,  1.2941
    2:   1.2313,  0.1460, -4.2000, -4.2000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  5.1715, 21.1347,  0.6132,  1.9073
    3:   1.0400,  0.1550, -3.8000, -3.8000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  3.9519, 25.0866,  0.5890,  2.4963
    4:  

   84:   2.0003,  0.1370,  0.3000, -3.8000,  3.8000,  4.1000,  8.9000,  8.9000, 13.0000,  7.6013,310.5156,  0.5206, 35.2675
   85:   1.8575,  0.1440,  3.8000,  3.8000,  0.0000,  0.0000, 13.0000, 13.0000, 13.0000, -0.7059,309.8098,  0.0000, 35.2675
   86:   1.8185,  0.1270, -1.7000, -1.7000,  0.0000,  0.0000, 13.0000, 13.0000, 13.0000,  3.0915,312.9013,  0.2159, 35.4834
   87:   1.9201,  0.1060,  1.3000,  1.3000,  0.0000,  0.0000, 13.0000, 13.0000, 13.0000, -0.2496,312.6516,  0.0000, 35.4834
   88:   2.1134,  0.1210,  1.3000,  1.3000,  0.0000,  0.0000, 13.0000, 13.0000, 13.0000, -0.2747,312.3769,  0.0000, 35.4834
   89:   2.4658,  0.1360, -1.0000, -1.0000,  0.0000,  0.0000, 13.0000, 13.0000, 13.0000,  2.4658,314.8427,  0.1360, 35.6194
   90:   2.9634,  0.1440, -5.5000, -3.5000,  0.0000, -2.0000, 13.0000, 13.0000, 11.0000, 10.3718,325.2145,  0.5040, 36.1234
   91:   3.6528,  0.1550, -8.1000, -4.9000,  0.0000, -3.2000, 11.0000, 11.0000,  7.8000, 17.8988,343.1133,  0.7595, 36.8829
   92:  

  160:   1.6417,  0.0700, 17.2000, 17.2000,  0.0000,  0.0000, 13.0000, 13.0000, 13.0000, -2.8237,617.8817,  0.0000, 56.4610
  161:   2.0018,  0.0950, 14.7000, 14.7000,  0.0000,  0.0000, 13.0000, 13.0000, 13.0000, -2.9427,614.9390,  0.0000, 56.4610
  162:   2.0204,  0.1170,  0.4000,  0.4000,  0.0000,  0.0000, 13.0000, 13.0000, 13.0000, -0.0808,614.8581,  0.0000, 56.4610
  163:   2.3369,  0.1040, -1.8000, -1.2000,  0.0000, -0.6000, 13.0000, 13.0000, 12.4000,  2.8042,617.6624,  0.1248, 56.5858
  164:   2.4341,  0.1030, -5.0000, -0.2000,  0.0000, -4.8000, 12.4000, 12.4000,  7.6000,  0.4868,618.1492,  0.0206, 56.6064
  165:   2.4007,  0.0990, -6.2000,  0.3000,  0.0000, -6.5000,  7.6000,  7.6000,  1.0000, -0.0720,618.0772,  0.0000, 56.6064
  166:   2.4107,  0.0860, -6.5000, -5.5000,  0.0000, -1.0000,  1.0000,  1.0000,  0.0000, 13.2587,631.3359,  0.4730, 57.0794
  167:   2.1943,  0.1030, -6.8000, -6.8000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, 14.9214,646.2572,  0.7004, 57.7798


In [8]:
print_price_summary(series_battery_DP)
logic_series_print(series_battery_DP)

The period is from 2022-06-19 00:00:00 to 2022-06-25 23:00:00
Cost for period:  610.0  DKK
Total emissions for period:  57.0  kg
Average cost per year is: 32009.0 DKK
Average emissions per year is: 3007.0 kg
Number of kwh purchased in the period: 408.79999999999995
Average number of kwh purchased per year: 21458.32814371257
Average number of kwh sold per year: 31841.05149700599
hour     price    eprice   yield    surplus  buy      charge   before   degrade  after    cost     pcumsum  emis     ecumsum 
    0:   1.5404,  0.1370, -5.5000, -5.5000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000,  8.4721,  8.4721,  0.7535,  0.7535
    1:   1.4134,  0.1020, -5.3000, -5.3000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000,  7.4911, 15.9632,  0.5406,  1.2941
    2:   1.2313,  0.1460, -4.2000, -4.2000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000,  5.1715, 21.1347,  0.6132,  1.9073
    3:   1.0400,  0.1550, -3.8000, -3.8000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000,  3.9519, 25.0866,  0.5890,  2.49

   85:   1.8575,  0.1440,  3.8000, -2.2000,  2.2000,  6.0000,  0.0000,  0.0000,  6.0000,  4.0865,275.0904,  0.3168, 32.7582
   86:   1.8185,  0.1270, -1.7000, -8.1000,  6.4000,  6.4000,  6.0000,  6.0000, 12.4000, 14.7301,289.8204,  1.0287, 33.7869
   87:   1.9201,  0.1060,  1.3000,  0.7000,  0.0000,  0.6000, 12.4000, 12.4000, 13.0000, -0.1344,289.6860,  0.0000, 33.7869
   88:   2.1134,  0.1210,  1.3000,  1.3000,  0.0000, -0.0000, 13.0000, 13.0000, 13.0000, -0.2747,289.4113,  0.0000, 33.7869
   89:   2.4658,  0.1360, -1.0000, -1.0000,  0.0000, -0.0000, 13.0000, 13.0000, 13.0000,  2.4658,291.8771,  0.1360, 33.9229
   90:   2.9634,  0.1440, -5.5000, -4.8000,  0.0000, -0.7000, 13.0000, 13.0000, 12.3000, 14.2242,306.1013,  0.6912, 34.6141
   91:   3.6528,  0.1550, -8.1000, -4.9000,  0.0000, -3.2000, 12.3000, 12.3000,  9.1000, 17.8988,324.0001,  0.7595, 35.3736
   92:   3.4436,  0.1650,-10.4000, -5.8000,  0.0000, -4.6000,  9.1000,  9.1000,  4.5000, 19.9726,343.9728,  0.9570, 36.3306
   93:  

  154:   1.3818,  0.0810,  8.3000,  8.3000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000, -1.1469,594.5511,  0.0000, 55.8878
  155:   1.3761,  0.0840, 10.1000,  3.1000,  0.0000,  7.0000,  0.0000,  0.0000,  7.0000, -0.4266,594.1245,  0.0000, 55.8878
  156:   1.4705,  0.0800, 17.6000, 24.6000,  0.0000, -7.0000,  7.0000,  7.0000,  0.0000, -3.6174,590.5071,  0.0000, 55.8878
  157:   1.3424,  0.1020, 19.3000, 13.3000,  0.0000,  6.0000,  0.0000,  0.0000,  6.0000, -1.7854,588.7217,  0.0000, 55.8878
  158:   1.2206,  0.0750, 18.7000, 11.7000,  0.0000,  7.0000,  6.0000,  6.0000, 13.0000, -1.4281,587.2936,  0.0000, 55.8878
  159:   1.3759,  0.0690, 21.3000, 21.3000,  0.0000, -0.0000, 13.0000, 13.0000, 13.0000, -2.9307,584.3629,  0.0000, 55.8878
  160:   1.6417,  0.0700, 17.2000, 17.2000,  0.0000, -0.0000, 13.0000, 13.0000, 13.0000, -2.8237,581.5392,  0.0000, 55.8878
  161:   2.0018,  0.0950, 14.7000, 14.7000,  0.0000, -0.0000, 13.0000, 13.0000, 13.0000, -2.9427,578.5965,  0.0000, 55.8878
  162:  